In [ ]:
import torch
import torch.nn as nn

# Vocabulary size dan dimensi embedding
vocab_size = 4  # Misal ada 4 kata dalam vocabulary
embedding_dim = 3  # Panjang vektor embedding

# Inisialisasi embedding layer
embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

# Input sequence (sebagai indeks dari kata-kata dalam vocabulary)
input_indices = torch.tensor([0, 1, 2])  # 'saya', 'makan', 'nasi'

# Mendapatkan vektor embedding untuk input
output = embedding(input_indices)

# Output
print(output)


In [ ]:
!pip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from google.colab import drive

# 1. Mount Google Drive untuk akses dataset
drive.mount('/content/drive')

# 2. Path ke dataset training
train_dir = '/content/drive/MyDrive/DATASET/train'

# 3. Preprocessing Data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

# Generator untuk data training
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Ukuran input VGG16
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Generator untuk data validation
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# 4. Memuat model VGG16 tanpa layer atas (top layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 5. Membekukan layer-layernya agar tidak dilatih
for layer in base_model.layers:
    layer.trainable = False

# 6. Menambahkan layer atas (top layers) untuk klasifikasi
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(5, activation='softmax')  # Sesuaikan dengan jumlah kelas
])

# 7. Compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 8. Training model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

# 9. Menyimpan model yang telah dilatih
model.save('/content/drive/MyDrive/DATASET/hasil/train/vgg16_model.h5')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 400 images belonging to 5 classes.
Found 100 images belonging to 5 classes.
58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/20
12/12 [==============================] - 21s 1s/step - loss: 5.1426 - accuracy: 0.2473 - val_loss: 1.1476 - val_accuracy: 0.5417
Epoch 2/20
12/12 [==============================] - 10s 788ms/step - loss: 2.2648 - accuracy: 0.4837 - val_loss: 0.9106 - val_accuracy: 0.5938
Epoch 3/20
12/12 [==============================] - 10s 817ms/step - loss: 0.8323 - accuracy: 0.6739 - val_loss: 0.8280 - val_accuracy: 0.6562
Epoch 4/20
12/12 [==============================] - 8s 671ms/step - loss: 0.7632 - accuracy: 0.6902 - val_loss: 0.7460 - val_accuracy: 0.7396
Epoch 5/20
12/12 [==============================] - 10s 812ms/step - loss: 0.7388 - accuracy: 0.6984 - val_loss: 0.6695 - val_accuracy: 0.7292
Epoch 6/20
12

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

# 1. Mount Google Drive untuk akses dataset
drive.mount('/content/drive')

# 2. Path ke dataset testing
test_dir = '/content/drive/MyDrive/DATASET/test'  # Ganti dengan path dataset pengujian Anda

# 3. Preprocessing Data untuk testing
test_datagen = ImageDataGenerator(rescale=1./255)  # Hanya rescaling untuk data pengujian

# Generator untuk data testing
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),  # Ukuran input VGG16
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Jangan shuffle data untuk pengujian
)

# 4. Memuat model yang telah dilatih
model = tf.keras.models.load_model('/content/drive/MyDrive/DATASET/hasil/train/vgg16_model.h5')

# 5. Evaluasi model menggunakan data testing
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

# 6. Melakukan prediksi pada data testing
predictions = model.predict(test_generator)
predicted_classes = tf.argmax(predictions, axis=1)  # Ambil kelas dengan probabilitas tertinggi

# 7. Menampilkan hasil prediksi
true_classes = test_generator.classes  # Kelas yang sebenarnya
class_labels = list(test_generator.class_indices.keys())  # Nama kelas

# Menampilkan beberapa hasil prediksi
for i in range(10):  # Tampilkan 10 prediksi pertama sebagai contoh
    print(f'Predicted class: {class_labels[predicted_classes[i]]}, True class: {class_labels[true_classes[i]]}')

# 8. Jika perlu, kamu bisa menghitung matriks kebingungan
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Matriks kebingungan
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

# Laporan klasifikasi
print("Classification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 215 images belonging to 5 classes.
6/6 [==============================] - 2s 201ms/step - loss: 0.4246 - accuracy: 0.8177
Test Loss: 0.4246181547641754, Test Accuracy: 0.8177083134651184
7/7 [==============================] - 7s 1s/step
Predicted class: anjing, True class: anjing
Predicted class: anjing, True class: anjing
Predicted class: domba, True class: anjing
Predicted class: anjing, True class: anjing
Predicted class: anjing, True class: anjing
Predicted class: anjing, True class: anjing
Predicted class: domba, True class: anjing
Predicted class: domba, True class: anjing
Predicted class: domba, True class: anjing
Predicted class: anjing, True class: anjing
Confusion Matrix:
[[23 10  4  5  1]
 [ 2 33  5  2  1]
 [ 0  1 42  0  0]
 [ 0  1  2 39  1]
 [ 0  0  0  0 43]]
Classification Report:
              precision    recall  f1-score   support

     